# Final Script for flags
Joins flags_auto, osu world and flag_additions


In [20]:
import requests
import json
import copy


In [21]:
def fetchData(api_url):
    try:
        response = requests.get(api_url)

        if response.status_code == 200:

            data = response.json()
            return data
        else:
            print(f"Request failed with status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)


## Clean flags auto leaving only the regions in osu world


In [22]:
flagsPathAuto = "./out/flags_auto.json"

with open(flagsPathAuto, "r", encoding="utf-8") as json_file:
    flagsAuto = json.load(json_file)


Add countries from osuworld

In [23]:
countries: dict = fetchData("https://osuworld.octo.moe/locales/en/countries.json")

In [24]:
for countryCode, country in countries.items():
    if countryCode not in flagsAuto:
        flagsAuto[countryCode] = {}
        flagsAuto[countryCode]['name'] = country
        flagsAuto[countryCode]['regions'] = {}
        print(f"Country {country} not found in flagsAuto.json")

Country Kazakhstan not found in flagsAuto.json
Country Singapore not found in flagsAuto.json
Country Turkey not found in flagsAuto.json
Country Taiwan not found in flagsAuto.json
Country Vietnam not found in flagsAuto.json


add regions from osu world

In [25]:
regions: dict = fetchData("https://osuworld.octo.moe/locales/en/regions.json")


In [26]:
flags: dict = copy.deepcopy(flagsAuto)

for countryCode, country in flagsAuto.items():
    osuWorldRegions: dict = regions.get(countryCode)
    if osuWorldRegions is None:
        flags.pop(countryCode, None)
        continue
    print(f"Country {country['name']} found in regions.json")
    for regionCode, region in country['regions'].items():
        osuWorldRegionKeys = osuWorldRegions.keys()
        if regionCode in osuWorldRegionKeys:
            region['name'] = osuWorldRegions[regionCode]
        else:
            flags[countryCode]['regions'].pop(regionCode, None)

    for osuWorldRegion in osuWorldRegions:
        if osuWorldRegion not in country['regions']:
            flags[countryCode]['regions'][osuWorldRegion] = {
                "name": osuWorldRegions[osuWorldRegion],
                "nativeName": "",
                "flag": ""
            }


Country Argentina found in regions.json
Country Australia found in regions.json
Country Austria found in regions.json
Country Belarus found in regions.json
Country Belgium found in regions.json
Country Brazil found in regions.json
Country Canada found in regions.json
Country Chile found in regions.json
Country China found in regions.json
Country Colombia found in regions.json
Country Czech Republic found in regions.json
Country Finland found in regions.json
Country France found in regions.json
Country Germany found in regions.json
Country Hungary found in regions.json
Country Indonesia found in regions.json
Country Italy found in regions.json
Country Japan found in regions.json
Country South Korea found in regions.json
Country Latvia found in regions.json
Country Malaysia found in regions.json
Country Mexico found in regions.json
Country Netherlands found in regions.json
Country Norway found in regions.json
Country Peru found in regions.json
Country Philippines found in regions.json
Co

Save

In [27]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path = "out/flags.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)


JSON data has been saved to out/flags.json


Reload data

In [28]:
flagsPath = "./out/flags.json"

with open(flagsPath, "r", encoding="utf-8") as json_file:
    flags = json.load(json_file)


## Add Manual changes to the automatic flags

In [39]:
flagsAdditionsPath = "./in/flags_additions.json"

with open(flagsAdditionsPath, "r", encoding="utf-8") as json_file:
    flagsAdditions = json.load(json_file)


In [30]:
import collections.abc

def update(d, u):
    for k, v in u.items():
        if isinstance(v, collections.abc.Mapping):
            d[k] = update(d.get(k, {}), v)
        else:
            d[k] = v
    return d

In [40]:
update(flags, flagsAdditions)

{'AR': {'name': 'Argentina',
  'nativeName': 'Argentina',
  'regions': {'AR-C': {'name': 'Buenos Aires',
    'nativeName': 'Buenos Aires',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/Bandera_de_la_Ciudad_de_Buenos_Aires.svg//128px-Bandera_de_la_Ciudad_de_Buenos_Aires.svg.png'},
   'AR-U': {'name': 'Chubut Province',
    'nativeName': 'Chubut',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/8/88/Bandera_de_la_Provincia_del_Chubut.svg'},
   'AR-E': {'name': 'Entre Ríos Province',
    'nativeName': 'Entre Ríos',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/5/5b/Bandera_de_la_Provincia_de_Entre_Ríos.svg'},
   'AR-Q': {'name': 'Neuquén Province',
    'nativeName': 'Neuquén',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/b/bc/Bandera_de_la_Provincia_de_Neuquén.svg'},
   'AR-R': {'name': 'Río Negro Province',
    'nativeName': 'Río Negro',
    'flag': 'https://upload.wikimedia.org/wikipedia/commons/5/5d/Bandera_de_la_Provinc

Save again


In [41]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path = "out/flags.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)


JSON data has been saved to out/flags.json


## Fill native names

Using Open Street Maps API: https://nominatim.openstreetmap.org/ui/search.html


In [33]:
urlSearch = "https://nominatim.openstreetmap.org/search.php?q=$1&countrycodes=$2&format=jsonv2"


def getSearchURL(regionName, countryCode):
    return urlSearch.replace("$1", regionName).replace("$2", countryCode)


urlReverseId = "https://nominatim.openstreetmap.org/details.php?osmtype=R&osmid=$1&format=json"


def getReverseIdURL(id):
    return urlReverseId.replace("$1", id)


In [34]:
def addResult(region, matchData):
    localName = matchData['name']
    if not localName:
        return False

    region["nativeName"] = localName
    print(localName)
    return True


def handleSearch(region, data):
    if data:
        it = (d for d in data if d.get("type") == "administrative")

        matchData = next(it, None)

        while matchData:
            suceed = addResult(region, matchData)
            if not suceed:
                matchData = next(it, None)
                if matchData is not None:
                    print("Trying next match")
            else:
                matchData = None
                return True

    else:
        print("No match data")
    return False


country: dict
for countryCode, country in flags.items():
    print(countryCode)

    nativeName = country.get('nativeName')
    if nativeName is None or nativeName == "":
        api_url = getSearchURL(country['name'], countryCode)
        data = fetchData(api_url)
        searchHandleResult = handleSearch(country, data)

        # Reorder regions to appear after nativeName
        regions = country['regions']
        del country['regions']
        country['regions'] = regions

    for regionCode, region in country['regions'].items():
        if region['nativeName'] != "":
            continue

        api_url = getSearchURL(regionCode, countryCode)
        data = fetchData(api_url)
        searchHandleResult = handleSearch(region, data)
        if not searchHandleResult:
            name = region['name']
            api_url = getSearchURL(name, countryCode)
            data = fetchData(api_url)
            searchHandleResult = handleSearch(region, data)


AR


Salta
Buenos Aires
San Luis
La Rioja
Santiago del Estero
Chaco
San Juan
Catamarca
La Pampa
Mendoza
Misiones
Formosa
Santa Fe
Tucumán
Corrientes
Córdoba
Jujuy
AU
AT
BY
BE
BR
CA
CL
Región de Arica y Parinacota
Región de Magallanes y de la Antártica Chilena
CN
CO
Amazonas
Antioquia
Arauca
Atlántico
Bolívar
Boyacá
Caldas
Caquetá
Casanare
Cauca
Cesar
Chocó
Córdoba
Cundinamarca
Guainía
Guaviare
Huila
La Guajira
Magdalena
Meta
Nariño
Norte de Santander
Putumayo
Quindío
Risaralda
Santander
Archipelago of Saint Andrew, Providence and Saint Catherine
Sucre
Tolima
Valle del Cauca
Vaupés
Vichada
CZ
Jihočeský kraj
Jihomoravský kraj
Karlovarský kraj
Královéhradecký kraj
Liberecký kraj
Moravskoslezský kraj
Olomoucký kraj
Pardubický kraj
Plzeňský kraj
Středočeský kraj
Ústecký kraj
Zlínský kraj
FI
Etelä-Karjala
Kymenlaakso
Lappi
Pirkanmaa
Österbotten
Pohjois-Pohjanmaa
Varsinais-Suomi
FR
DE
Bayern
HU
ID
Papua Tengah
Maluku
Papua Selatan
Papua Barat Daya
IT
JP
KR
세종
LV
Kurzeme
Latgale
Vidzeme
Zemgale
MY


In [35]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()
file_path = "out/flags.json"
with open(file_path, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", file_path)


JSON data has been saved to out/flags.json


## Reload data


In [36]:
flagsPath = "./out/flags.json"

with open(flagsPath, "r", encoding="utf-8") as json_file:
    flags = json.load(json_file)


In [37]:
[region['name'] for region in flags["ES"]["regions"].values()]


['Andalusia',
 'Aragon',
 'Asturias',
 'Basque Country',
 'Canary Islands',
 'Cantabria',
 'Castilla–La Mancha',
 'Castile and León',
 'Catalonia',
 'Extremadura',
 'Galicia',
 'Community of Madrid',
 'Region of Murcia',
 'Navarre',
 'Valencian Community',
 'Ceuta',
 'Melilla',
 'Balearic Islands',
 'La Rioja']

Countries missing regions

In [38]:
printJsons = False

for countryCode, country in flags.items():
    regions = country["regions"]
    flagsFilter = [region['flag'] for region in regions.values()]
    containsEmptyFlag = any(flag == "" for flag in flagsFilter)
    if containsEmptyFlag:
        print("-", country["name"])
        if printJsons:
            print(json.dumps(country, indent=4, ensure_ascii=False))

- Philippines
- Romania
- Kazakhstan
- Turkey
